# MODULES

In [5]:
import panel as pn
import panel.widgets as pnw
pn.extension('plotly') #para que nos despliegue en nuestro notebook, activamos la extension de panel

import pandas as pd
import numpy as np

import plotly.graph_objects as go

import math

# CLASSES

In [6]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# FUNCTIONS

## Euclidian Distance

In [7]:
def dist_eucl(p1, p2):
  dist_E = math.sqrt(sum([(x - y)** 2 for x, y in zip(p1, p2)]))
  return dist_E

# TRAJECTORIES

## Brownian Motion (BM)

In [8]:
def BM(n_steps = 1000, speed = 6, s_x_pos = 0, s_y_pos = 0):

  #Init velocity vector, La clase Vec2d nos sirve como herramienta para definir el vector de velocidad que nos dice en que direccion y a que rapidez nos estamos moviendo
  velocity = Vec2d(speed,0) #<- cuando ya tenemos la rapidez en la que nos queremos mover, podemos definir este vector de velocidad con 
  #una instancia de la clase vec2d q inicia como un vector horizontal teniendo x=6 & y=0 

  #Init DataFrame
  BM_df = pd.DataFrame(columns= ['x_pos', 'y_pos'])

  #DF aux para cargar la pos inicial
  temp_df = pd.DataFrame([{'x_pos': s_x_pos, 'y_pos': s_y_pos}])# le mandamos un diccionario con los valores de la posicion x & y
  
  #concatenamos el df actual y el DF aux(temp), ignore_index=True para que no tome en cuenta los indices y no genere uno nuevo que inicie en 0 
  BM_df = pd.concat([BM_df, temp_df], ignore_index=True)#mete la primer posicion del agente
  
  #para poblarlo generamos los valores aleatorios de los giros
  for i in range(n_steps-1):
    #generamos los giros para BM a partir de una distribucion uniforme
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi) 
    velocity = velocity.rotated(turn_angle) #giramos el vector de velocidad segun el valor aleatorio asignado por turn angle 

    temp_df = pd.DataFrame([{'x_pos': BM_df.x_pos[i]+velocity.x, 'y_pos':BM_df.y_pos[i]+velocity.y}])
    BM_df= pd.concat([BM_df, temp_df], ignore_index=True)

  #return DF
  return BM_df


### BM Trajectory 3d

In [9]:
def plot_trajBM(nSteps, speed, xInitPos, yInitPos, rw_df):#recibe los valores de los widgets n_steps y s_x_pos
  fig_traj_rw = go.Figure()#def instancia de la tray

  #rw_df = BM(nSteps, speed, xInitPos, yInitPos)#df, trayectoria con n pasos generada con la funcion bm_2d, y especificamos 

  fig_traj_rw.add_trace(#definimos la trayectoria haciendo uso de trazos
      go.Scatter3d(x = rw_df.x_pos, #Le mandamos toda la columna x_pos
                   y = rw_df.y_pos,
                   z = rw_df.index, #index como aux para z
                   marker = dict(size=2),
                   line = dict(color = 'red', width=2),
                   mode = 'lines',
                   name = f'steps = {nSteps}', #nombre dinamico, el num de pasos es = a 
                   showlegend = True
                   )
  )
  return fig_traj_rw

## PL BM

In [10]:
def BMPL(BMTray):
  BMTrayArr = BMTray
  #Calculamos la distribucion de Path length para las trayectorias y lo guardamos en un arreglo np

  #La distancia recorrida va a ser de 0 a 1, 0-2, 0-3 susesivamente, al final veremos cual es la distancia que recorrio el 
  #agente conforme avanzo en el tiempo a cada segundo
  dis_BM_3 = np.array([distance.euclidean(BMTrayArr.iloc[i-1], BMTrayArr.iloc[i])for i in range(1,BMTrayArr.shape[0])])
  #Se calcula la dist entre un paso y el que le sigue usando la func euclidean(punto1, punto2), el for recorre todo el df utizando
  #un aux como rango desde 1 y el ult valor es = a la long de nuestro vector,poreso ponemos el df y el primer elem[0] que nos da el numero de renglones

  #Calculamos la distancia acumulad(PL)
  pl_BM_3 = np.cumsum(dis_BM_3) #(recibe como argumento el vector y nos devuelve la sumatoria de todos los elementos

  #Inicializamos, Graficamos los trazos de PL
  fig_path_length = go.Figure()

  #trace BM_3
  fig_path_length.add_trace(go.Scatter(
      x = np.arange(len(pl_BM_3)),# aux para ver como avanza PL(longitud de(x)), genera un vector con valores desde 0 hasta x
      y = pl_BM_3, #arreglo calculado PL
      marker = dict(size=2),
      line = dict(width=2),
      mode = 'lines',
      name = 'path_length_BM',
      showlegend = True
  ))

  return fig_path_length

## MSD BM

In [11]:
def BMMSD(bm_2d):
  #Init dataframe
  MSD_BM= pd.DataFrame(columns= ['MSD_BM'])

  temp_df = pd.DataFrame([{'MSD_BM': 0}])

  # MSD for BM_2d
  for tau in range(1,bm_2d.shape[0]):
    displacement_vec = np.array([dist_eucl(bm_2d.iloc[i-tau], bm_2d.iloc[i])**2#Calculo de c/u de las ventanas
                    for i in range(tau, bm_2d.shape[0],1) ])#Barrido de la trayectoria

    MSD_BM_ecuacion = 1/(bm_2d.shape[0]-tau)*(np.sum(displacement_vec))#complemento de la ecuacion MSD
    temp_df = pd.DataFrame([{'MSD_BM': MSD_BM_ecuacion}])
    MSD_BM = pd.concat([MSD_BM,temp_df], ignore_index=True)

  fig_MSD_BM = go.Figure()

  #first BM_6
  fig_MSD_BM.add_trace(go.Scatter(
      x = MSD_BM.index, #ya q estamos usando df utilizar el index como aux######################bm_ed
      y = MSD_BM.MSD_BM, # columna BM ##################################################
      marker = dict(size=2),
      line = dict(width=2),
      mode = 'lines',
      name = 'MSD BM 6',
      showlegend = True
  ))
  return fig_MSD_BM

# PANEL WIDGETS

In [12]:
#Sliders
nSteps = pnw.IntSlider(name='Number of steps', width=300, value=50, step=10, start=0, end=1000)
speed = pnw.IntSlider(name='Speed', width=300, value=3, step=1, start=0, end=6)

#IntImput
xInitPos = pnw.IntInput(name='xInitPos', width=100, value=0, step=1,start=1, end=50)
yInitPos = pnw.IntInput(name='yInitPos', width=100, value=0, step=1,start=1, end=50)

#rw_df = 0

@pn.depends(nSteps, speed, xInitPos, yInitPos) #agregamos el decorador para q escuche los widgets en orden
def plotBM(nSteps, speed, xInitPos, yInitPos):
  rw_df = BM(nSteps, speed, xInitPos, yInitPos)
  return plot_trajBM(nSteps, speed, xInitPos, yInitPos, rw_df)

#Select
metricsType = pnw.Select(name='Metrics type', value='MSD', options=['MSD', 'PL'])
@pn.depends(metricsType, nSteps, speed, xInitPos, yInitPos)
def plotBMTray(rw_df, nSteps, speed, xInitPos, yInitPos):
  rw_df = BM(nSteps, speed, xInitPos, yInitPos)
  if(metricsType.value == 'MSD'):
    return BMMSD(rw_df)
  else:
    return BMPL(rw_df)

radioButtons = pn.widgets.RadioButtonGroup(options=['BM', 'CRW', 'LF'])
@pn.depends(radioButtons,speed,nSteps,xInitPos,yInitPos)
def params(x):
  if x == 'BM':
    bmRowCol = pn.Row(
        pn.Column(
          pn.Row('',nSteps),
          pn.Row(
              pn.Column('',xInitPos),
              pn.Spacer(width=50),
              pn.Column('',yInitPos)
          ),
          pn.Row('',speed),
          pn.Row('',metricsType)
        ),
        pn.Column(
            pn.Row(plotBM)
        ),
        pn.Column(
            pn.Row(plotBMTray)
        )
    )
    return bmRowCol
  elif x == 'CRW':
    bmRowCol = pn.Column(
        pn.Row('',nSteps),
        pn.Row(
        pn.Column('',xInitPos),
        pn.Spacer(width=50),
        pn.Column('',yInitPos)
    ),
      pn.Row('',speed)  
    )
    return bmRowCol 
  elif x == 'LF':
    bmRowCol = pn.Column(
        pn.Row('',nSteps),
        pn.Row(
        pn.Column('',xInitPos),
        pn.Spacer(width=50),
        pn.Column('',yInitPos)
    ),
      pn.Row('',speed)  
    )
    return bmRowCol 

# create interaction between widget and function
pn.interact(params, x=radioButtons)


Traceback (most recent call last):
 File "C:\Users\Sonia\.conda\envs\Topico_Industria\lib\site-packages\pyviz_comms\__init__.py", line 346, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\Sonia\.conda\envs\Topico_Industria\lib\site-packages\panel\viewable.py", line 292, in _on_msg
 doc.unhold()
 File "C:\Users\Sonia\.conda\envs\Topico_Industria\lib\site-packages\bokeh\document\document.py", line 799, in unhold
 self.callbacks.unhold()
 File "C:\Users\Sonia\.conda\envs\Topico_Industria\lib\site-packages\bokeh\document\callbacks.py", line 396, in unhold
 self.trigger_on_change(event)
 File "C:\Users\Sonia\.conda\envs\Topico_Industria\lib\site-packages\bokeh\document\callbacks.py", line 373, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "C:\Users\Sonia\.conda\envs\Topico_Industria\lib\site-packages\bokeh\document\callbacks.py", line 408, in invoke_with_curdoc
 return f()
 File "C:\Users\Sonia\.conda\envs\Topico_Industria\lib\site-packages\bokeh\util\callback_manager.py", line 191, in invoke
 callback(attr, old, new)
 File "C:\Users\Sonia\.conda\envs\Topico_Industria\lib\site-packages\panel\reactive.py", line 392, in _comm_change
 self._schedule_change(doc, comm)
 File "C:\Users\Sonia\.conda\envs\Topico_Industria\lib\site-packages\panel\reactive.py", line 376, in _schedule_change
 self._change_event(doc)
 File "C:\Users\Sonia\.conda\envs\Topico_Industria\lib\site-packages\panel\reactive.py", line 370, in _change_event
 self._process_events(events)
 File "C:\Users\Sonia\.conda\envs\Topico_Industria\lib\site-packages\panel\reactive.py", line 315, in _process_events
 self.param.update(**self_events)
 File "C:\Users\Sonia\.conda\envs\Topico_Industria\lib\site-packages\param\parameterized.py", line 1898, in update
 self_._batch_call_watchers()
 File "C:\Users\Sonia\.conda\envs\Topico_Industria\lib\site-packages\param\parameterized.py", line 2059, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "C:\Users\Sonia\.conda\envs\Topico_Industria\lib\site-packages\param\parameterized.py", line 2021, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "C:\Users\Sonia\.conda\envs\Topico_Industria\lib\site-packages\panel\param.py", line 794, in _replace_pane
 new_object = self.eval(self.object)
 File "C:\Users\Sonia\.conda\envs\Topico_Industria\lib\site-packages\panel\param.py", line 777, in eval
 return function(*args, **kwargs)
 File "C:\Users\Sonia\.conda\envs\Topico_Industria\lib\site-packages\param\parameterized.py", line 407, in _depends
 return func(*args, **kw)
 File "C:\Users\Sonia\AppData\Local\Temp\ipykernel_22520\1900941493.py", line 24, in plotBMTray
 return BMPL(rw_df)
 File "C:\Users\Sonia\AppData\Local\Temp\ipykernel_22520\3935980546.py", line 7, in BMPL
 dis_BM_3 = np.array([distance.euclidean(BMTrayArr.iloc[i-1], BMTrayArr.iloc[i])for i in range(1,BMTrayArr.shape[0])])
 File "C:\Users\Sonia\AppData\Local\Temp\ipykernel_22520\3935980546.py", line 7, in <listcomp>
 dis_BM_3 = np.array([distance.euclidean(BMTrayArr.iloc[i-1], BMTrayArr.iloc[i])for i in range(1,BMTrayArr.shape[0])])
NameError: name 'distance' is not defined

Column
    [0] Column
        [0] RadioButtonGroup(options=['BM', 'CRW', 'LF'], value='BM')
    [1] Row
        [0] Row
            [0] Column
                [0] Row
                    [0] Markdown(str)
                    [1] IntSlider(end=1000, name='Number of steps', step=10, value=50, width=300)
                [1] Row
                    [0] Column
                        [0] Markdown(str)
                        [1] IntInput(end=50, name='xInitPos', start=1, width=100)
                    [1] Spacer(width=50)
                    [2] Column
                        [0] Markdown(str)
                        [1] IntInput(end=50, name='yInitPos', start=1, width=100)
                [2] Row
                    [0] Markdown(str)
                    [1] IntSlider(end=6, name='Speed', value=3, width=300)
                [3] Row
                    [0] Markdown(str)
                    [1] Select(name='Metrics type', options=['MSD', 'PL'], value='MSD')
            [1] Column
                [0] Row
                    [0] ParamFunction(function)
            [2] Column
                [0] Row
                    [0] ParamFunction(function)